In [1]:
import speech_recognition as sr

In [2]:
from difflib import SequenceMatcher

def similarity_percentage(str1, str2):
    from difflib import SequenceMatcher
    matcher = SequenceMatcher(None, str1, str2)
    return round(matcher.ratio() * 100, 2)

In [3]:
def evaluate_question(question):
    """
    Function to evaluate whether the recognized question is acceptable or not.
    Returns "Accept" if the question matches predefined questions with at least 40% similarity,
    otherwise returns "Reject".
    """
    # Dictionary of predefined questions
    predefined_questions = {
        "What does NLP stand for?": "natural language processing",
        "Can you name a popular NLP library in Python?": "NLTK",
        "What is the process of converting text into numerical representations called?": "Text Vectorization",
        "What is the name of the state-of-the-art language model developed by OpenAI?": "GPT-3",
    }

    # Check if the question is similar to any predefined question
    max_similarity_question = max(predefined_questions.keys(), key=lambda q: similarity_percentage(question.lower(), q.lower()))
    similarity_percent_question = similarity_percentage(question.lower(), max_similarity_question.lower())

    # If similarity percentage for question is less than 40, reject the question
    if similarity_percent_question < 40:
        print(f"Sorry, the question doesn't match any predefined question. ({similarity_percent_question}% match)")
        return "Reject"

    print(f"Question accepted. ({similarity_percent_question}% match)")
    return "Accept"

In [17]:
def evaluate_answer(question, answer):
    """
    Function to evaluate the correctness of the answer for the given question.
    Returns "Correct" if the answer matches the predefined answer with at least 40% similarity,
    otherwise returns "Incorrect". If the similarity percentage is less than 40%, it also returns
    the correct answer for the corresponding question.
    """
    # Dictionary of questions and their correct answers
    correct_answers = {
        "What does NLP stand for?": "Natural Language Processing",
        "Can you name a popular NLP library in Python?": "NLTK",
        "What is the process of converting text into numerical representations called?": "Text Vectorization",
        "What is the name of the state-of-the-art language model developed by OpenAI?": "GPT-3",
    }

    # Case-insensitive matching for the question
    question_lower = question.lower()

    # Get the matching question from the dictionary, if any
    matched_question = next((q for q in correct_answers.keys() if question_lower in q.lower()), None)

    if matched_question:
        # Get the correct answer for the matched question
        correct_answer = correct_answers[matched_question].lower()

        # Check similarity percentage for answer
        similarity_percent_answer = similarity_percentage(answer.lower(), correct_answer)

        # If similarity percentage for answer is less than 90, reject the answer and provide correct answer
        if similarity_percent_answer < 90:
            print(f"Incorrect answer. The correct answer is: {correct_answers[matched_question]}. Please try again. ({similarity_percent_answer}% match)")
            return "Incorrect", correct_answers[matched_question]
        else:
            print(f"Correct answer! Well done. ({similarity_percent_answer}% match)")
            return "Correct", None
    else:
        print("Question not recognized.")
        return "Question not recognized", None


In [8]:
def main():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    while True:
        print("Listening for question...")
        with microphone as source:
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source)

        try:
            print("Processing question...")
            question = recognizer.recognize_google(audio)
            print("Your question:", question)
            response = evaluate_question(question)

            if response == "Accept":
                print("Listening for answer...")
                with microphone as source:
                    recognizer.adjust_for_ambient_noise(source)
                    audio = recognizer.listen(source)

                try:
                    print("Processing answer...")
                    answer = recognizer.recognize_google(audio)
                    print("Your answer:", answer)
                    response = evaluate_answer(question, answer)
                    print("Response:", response)
                except sr.UnknownValueError:
                    print("Sorry, I couldn't understand what you said for the answer.")
                except sr.RequestError as e:
                    print("Error:", e)

        except sr.UnknownValueError:
            print("Sorry, I couldn't understand what you said for the question.")
        except sr.RequestError as e:
            print("Error:", e)

        # Check if the user said 'stop', to break the loop
        if 'stop' in question.lower() or 'stop' in answer.lower():
            print("Stopping...")
            break


In [11]:
if __name__ == "__main__":
    main()

Listening for question...
Processing question...
Your question: NLP stand for
Question accepted. (70.27% match)
Listening for answer...
Processing answer...
Your answer: nature
Incorrect answer. The correct answer is: Natural Language Processing. Please try again. (36.36% match)
Response: ('Incorrect', 'Natural Language Processing')
Listening for question...
Processing question...
Your question: what does NLP stands for
Question accepted. (95.83% match)
Listening for answer...
Processing answer...
Your answer: language process
Question not recognized.
Response: ('Question not recognized', None)
Listening for question...
Processing question...
Your question: what does NLP stand for
Question accepted. (97.87% match)
Listening for answer...
Processing answer...
Your answer: natural language processing
Correct answer! Well done. (100.0% match)
Response: ('Correct', None)
Listening for question...
Processing question...
Sorry, I couldn't understand what you said for the question.
Listening 